## 02 EDA  
(introduction here)

#### Skip Ahead:  
[Read and Concatenate](#Read-and-Concatenate)  
[Exploration](#Exploration)  
[header3]

In [2]:
#imports
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import re
#week 3
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#week 4
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
#week 5
import requests
from bs4 import BeautifulSoup

### Read and Concatenate

In [3]:
xb = pd.read_csv('./sub_data/Xbox_10k.csv') #read in xbox data
xb.shape

(10005, 8)

In [4]:
ps = pd.read_csv('./sub_data/Playstation_10k.csv')
ps.shape

(10021, 8)

In [5]:
df = pd.concat([xb, ps], ignore_index=True) #make 1 dataframe with all the posts from r/xbox and r/playstatino

In [6]:
df.shape

(20026, 8)

In [7]:
df.head()

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url
0,xbox,jc40iu,ImmaPoodle,1,So watch dogs legion is coming out soon and wi...,Ubisoft vs Rockstar,1.0,https://www.reddit.com/r/xbox/comments/jc40iu/...
1,xbox,jc3ty3,jaaytf_,2,hey so my xbox one is turning on and instantly...,xbox one problems:),1.0,https://www.reddit.com/r/xbox/comments/jc3ty3/...
2,xbox,jc3tg4,Matty_Rts,0,"For context, I purchased game pass on Xbox One...",My Preorder download for Beyond Light Disappea...,1.0,https://www.reddit.com/r/xbox/comments/jc3tg4/...
3,xbox,jc39t3,Rorydo44,2,I had $50 worth of gift cards in my account an...,Missing gift cards,1.0,https://www.reddit.com/r/xbox/comments/jc39t3/...
4,xbox,jc37vv,NonsenseText,0,Hi everyone! \n\nI would like to learn more ab...,Advice for Xbox online security?,1.0,https://www.reddit.com/r/xbox/comments/jc37vv/...


In [8]:
df.tail()

,subreddit,id,author,num_comments,selftext,title,upvote_ratio,url
20021,playstation,hf6ijf,IamXnoskillzX,3,I pre ordered the game awhile ago and now it s...,Did Sony Pull Ghosts Of Tsushima From Amazon?,1.00,https://www.reddit.com/r/playstation/comments/...
20022,playstation,hf5tl0,[deleted],0,NaN,Need help from PlayStation fans!,1.00,https://www.reddit.com/r/playstation/comments/...
20023,playstation,hf5rp1,ngocan2121,5,im just a casual player so here is my list and...,"Looking for sad story game with nice gameplay,...",0.75,https://www.reddit.com/r/playstation/comments/...
20024,playstation,hf58u2,SwingDingeling,10,Or evwn 8K?,Will every PS5 game be in 4K?,0.50,https://www.reddit.com/r/playstation/comments/...
20025,playstation,hf50cd,[deleted],1,[deleted],[HELP] somebody is using my email for a PlaySt...,0.50,https://www.reddit.com/r/playstation/comments/...


In [10]:
df['subreddit'].value_counts() #looks right

playstation    10021
xbox           10005
Name: subreddit, dtype: int64

### Exploration

My first task is going to be distinguishing between which subreddit posts came from. However, in the future I also want to look deeper into which words are popular in the playstation subreddit to find what those gamers value and potentially assist Microsoft's marketing team in winning over more customers. With that target in mind, I'll make the playstation reddit my "1" or target, and the xbox subreddit my "0".

In [11]:
df['subreddit'] = df['subreddit'].map({'xbox':0, 'playstation':1})

Next, I want to create a new column that concatenates the 'selftext' and 'title' column text together in order to be able to include all of their combined text in my model. First I need to check out my nans.

In [15]:
df.isnull().sum()

subreddit         0
id                0
author            0
num_comments      0
selftext        971
title             0
upvote_ratio      0
url               0
dtype: int64

Okay, 971 entries where self_text is empty, but everything else lookg good!

In [20]:
df['selftext'].value_counts().sort_values(ascending=False).head(10)

[deleted]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              325
Title                                                                                        

So there is a high number of [deleted] selftext items, and then some that just say 'Title', and then some that use punctuation (either '.' or '?'). I wonder if I have any duplicate posts.

In [22]:
df['title'].value_counts()

I need help                                                                                    20
Help                                                                                           14
Latest update                                                                                  12
Quick question                                                                                 12
Backwards compatibility                                                                        10
                                                                                               ..
Who are your favourite PlayStation characters?                                                  1
Updates delete games?                                                                           1
Will Watchdog legions be availble on gamepass? I do not want to pay full price of the game.     1
Family Gaming Help                                                                              1
Will I still be able

There do appear to be some duplicate titles, but they're a little generic so I can't be sure if it's a true repost or just a commonly invoked title for similar issues. I'm going to write a loop to test if a post has the same title, selftext, and author.

In [27]:
df[['author', 'title', 'selftext']]

,author,title,selftext
0,ImmaPoodle,Ubisoft vs Rockstar,So watch dogs legion is coming out soon and wi...
1,jaaytf_,xbox one problems:),hey so my xbox one is turning on and instantly...
2,Matty_Rts,My Preorder download for Beyond Light Disappea...,"For context, I purchased game pass on Xbox One..."
3,Rorydo44,Missing gift cards,I had $50 worth of gift cards in my account an...
4,NonsenseText,Advice for Xbox online security?,Hi everyone! \n\nI would like to learn more ab...
...,...,...,...
20021,IamXnoskillzX,Did Sony Pull Ghosts Of Tsushima From Amazon?,I pre ordered the game awhile ago and now it s...
20022,[deleted],Need help from PlayStation fans!,NaN
20023,ngocan2121,"Looking for sad story game with nice gameplay,...",im just a casual player so here is my list and...
20024,SwingDingeling,Will every PS5 game be in 4K?,Or evwn 8K?


In [39]:
df['author'][0]

'ImmaPoodle'

In [45]:
for n in range(len(df)):
    for r in range(len(df)):
        if df['author'][n] == df['author'][(n+1)+(r-n)] and df['title'][n] == df['title'][(n+1)+(r-n)] and df['selftext'][n] == df['selftext'][(n+1)+(r-n)]:
            print(f'Index{n} is a duplicate of index{n+1+(r-n)}')
                  

Index0 is a duplicate of index10006


KeyError: 20026